In [67]:
import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
sys.path.append('../src/')
from evaluate import get_results
from omegaconf import OmegaConf
from main import load_data
from model import AEDist
import numpy as np
import os
import glob
import torch

def prepare_dm_data(cfg, save_path='../dm_data/', folder_path="../src/wandb/"):
    folder_list = glob.glob(f"{folder_path}*{run.id}*")
    ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
    ckpt_path = ckpt_files[0]
    data_path = os.path.join(cfg.data.root, cfg.data.name + cfg.data.filetype)
    data = np.load(data_path, allow_pickle=True)
    # model = AEDist(dim=50, emb_dim=10)
    # model.load_from_checkpoint(ckpt_path)
    model = AEDist.load_from_checkpoint(ckpt_path)
    model.eval()
    x_all = torch.tensor(data['data'], dtype=torch.float32)
    x_pred, z_pred = model(x_all)
    x_pred = x_pred.detach().cpu().numpy()
    z_pred = z_pred.detach().cpu().numpy()
    save_name = f'{save_path}/{cfg.data.name}_{cfg.model.emb_dim}_dm.npz'
    np.savez(save_name, data=z_pred, train_mask=data['is_train'])


In [14]:
wandb.login()
api = wandb.Api()

# Specify your entity, project, and sweep ID
entity = "xingzhis"
project = "dmae"
sweep_id = 'j7tcyoop'

# Fetch the sweep
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")

In [49]:
# Initialize an empty list to store run data
runs_data = []

# Iterate through each run in the sweep
for run in sweep.runs:
    # Extract metrics and configs
    metrics = run.summary._json_dict
    configs = run.config
    
    # Combine metrics and configs, and add run ID
    combined_data = {**metrics, **configs, "run_id": run.id}
    
    # Append the combined data to the list
    runs_data.append(combined_data)

# Create a DataFrame from the runs data
df = pd.DataFrame(runs_data)

In [75]:
dims = df['emb_dim'].unique()
data_names = df['data.name'].unique()
dfs = []
for dim in dims:
    for data_name in data_names:
        ids = df[(df['data.name'] == data_name) & (df['emb_dim'] == dim)].sort_values(by=['validation/reconstr_loss', 'validation/dist_accuracy'], ascending=[True, False]).iloc[0,:][['data.name', 'emb_dim', 'validation/reconstr_loss', 'validation/dist_accuracy', 'run_id']]
        run_id = ids['run_id']
        run = [run for run in sweep.runs if run.id == run_id][0]
        cfg = OmegaConf.create(run.config)
        prepare_dm_data(cfg)
        dfs.append(ids)

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/dmae/notebooks/../src/model.py:281: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer('mean', torch.tensor(mean, dtype=torch.float32), persistent=True)
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/dmae/notebooks/../src/model.py:282: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer('std', torch.tensor(std, dtype=torch.float32), persistent=True)
/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'activation_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore t

In [1]:
df_name = pd.concat(dfs, axis=1).T.sort_values(['data.name', 'emb_dim'])
df_name.to_csv('../dm_data/differnt_dims.csv', index=False)

NameError: name 'pd' is not defined

In [77]:
df_name['sweep'] = sweep_id

In [78]:
df_name

,data.name,emb_dim,validation/reconstr_loss,validation/dist_accuracy,run_id,sweep
28,eb_subset_all,5,1.428262,0.866203,9xtkqnkl,j7tcyoop
27,eb_subset_all,15,1.31582,0.867569,u98snjwi,j7tcyoop
25,eb_subset_all,20,1.294534,0.879244,9sapq4h8,j7tcyoop
3,eb_subset_all,25,1.277464,0.875017,m07wclny,j7tcyoop
16,eb_subset_all,30,1.27806,0.880826,p14vurq1,j7tcyoop
74,sea_ad_gaba_all,5,0.657298,0.918041,najohsy0,j7tcyoop
56,sea_ad_gaba_all,15,0.638394,0.927088,m2874yfn,j7tcyoop
78,sea_ad_gaba_all,20,0.639052,0.937962,4se1cs5d,j7tcyoop
52,sea_ad_gaba_all,25,0.626545,0.937596,islnpqz2,j7tcyoop
71,sea_ad_gaba_all,30,0.647009,0.932049,cbgj4rf2,j7tcyoop
